In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime



# Define the URL and tickers

market_insiderUrl = 'https://markets.businessinsider.com/stocks/'
tickers = ['AMZN','TSLA','META','MSFT','GOOG','NVDA','AAPL']

news_tables = {}
for ticker in tickers:
    news_table=[]
    for i in range(1,6):
        url = market_insiderUrl + ticker + '-stock/news?p='+str(i)



        # Use BeautifulSoup to parse the loaded page
        print(url)
        response = requests.get(url)
        response.raise_for_status()  # This will raise an exception for HTTP errors

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        news_table.extend(soup.findAll('a', class_='news-link'))
    news_tables[ticker] = news_table

print(len(news_tables['MSFT']))




https://markets.businessinsider.com/stocks/AMZN-stock/news?p=1
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=2
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=3
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=4
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=5
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=1
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=2
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=3
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=4
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=5
https://markets.businessinsider.com/stocks/META-stock/news?p=1
https://markets.businessinsider.com/stocks/META-stock/news?p=2
https://markets.businessinsider.com/stocks/META-stock/news?p=3
https://markets.businessinsider.com/stocks/META-stock/news?p=4
https://markets.businessinsider.com/stocks/META-stock/news?p=5
https://markets.businessinsider.com/stocks/MSFT-stock/n

In [3]:
print(len(news_tables['AMZN']))

250


In [5]:
parsed_data = []

p_data =[];



for ticker, news_table in news_tables.items():
    print(ticker)
    for elem in news_table:

        try:
            articleInfo = ''
            if elem:
               
                link = elem.get('href')
                newUrl = 'https://markets.businessinsider.com' + link 

                # Use Selenium to navigate to the new URL
                response = requests.get(newUrl)
                response.raise_for_status()  # This will raise an exception for HTTP errors

                # Parse the HTML using BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')
                h1tag = soup.find('h1',class_='article-title')
                title = h1tag.text
                
                date = soup.find('span',class_='news-post-quotetime warmGrey')
                print(date)
                date = date.text
                date = date.split(',')
                date = date[0] + ','+ date[1];
                
                date_format = "%b. %d, %Y"

                # Convert the string to a datetime object
                date_object = datetime.strptime(date, date_format)
                iso_format_date = date_object.isoformat()


                # Format the date in the standard ISO 8601 format

                content = soup.find('div', class_='col-xs-12 news-content no-padding')
                irrelevant_phrases = ["InvestorPlace - Stock Market News","Shutterstock","Source:","See also", "Read next", "Read also", "TipRanks has tracked 36,000", "Now Read", "Read now"]

                if content:
                # Extract all paragraphs, ignoring nested formatting tags by getting the .text
                    p_tags = content.find_all('p')
                    restIrelevant = False
                    pList=[]
                    for p in p_tags:
                        relevant = True
                        
                        if(len(p.text.strip())==0):
                            continue;
                        

                        if(len(p.text.strip().split())<7):
                            relevant=False
                        if(p.text.strip()=="InvestorPlace - Stock Market News, Stock Advice & Trading Tips"):
                            relevant=False
                        if(restIrelevant):
                            relevant = False
                            
                        if 'InvestorPlace.com'.lower() in p.text.strip().lower():
                            relevant = False
                            restIrelevant=True
            
                        for phrase in irrelevant_phrases:
                            if phrase.lower() in p.text.strip().lower(): 
                                relevant = False
                                break
                        pList.append(p.text.strip())
                          
                        p_data.append([ticker,link,p.text.strip(),relevant])
                    
                    articleInfo = ' '.join(p.text.strip() for p in p_tags if len(p.text.strip()) != 0)

                parsed_data.append([ticker, title,iso_format_date, link, articleInfo,pList])
                
        except Exception as e:
            print(f"An error occurred while fetching or parsing the article: {e}")
            continue  # Skip to the next loop iteration if an error occurs

df = pd.DataFrame(parsed_data, columns=['ticker', 'title','date', 'link', 'articleInfo','paragraphList'])
p_df = pd.DataFrame(p_data, columns=['ticker','link','paragraph','relevant'])

# print(df.head())


AMZN
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 02:36 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 01:55 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 01:22 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 09:53 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 09:40 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 09:33 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 09:21 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 08:55 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 08:53 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 08:23 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 06:50 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 05:37 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 18, 2024, 04:15 AM</span>
<span class="news-post-quotetime 

In [ ]:
print(df['articleInfo'])

0     The commercial real estate industry continues ...
1     InvestorPlace - Stock Market News, Stock Advic...
2     InvestorPlace - Stock Market News, Stock Advic...
3     As the first-quarter (Q1) 2024 earnings season...
4     J.P. Morgan analyst Doug Anmuth maintained a B...
5     InvestorPlace - Stock Market News, Stock Advic...
6     Analysts have been eager to weigh in on the Co...
7     Needham analyst Laura Martin initiated coverag...
8     As the tech sector braces for its upcoming ear...
9     InvestorPlace - Stock Market News, Stock Advic...
10    InvestorPlace - Stock Market News, Stock Advic...
11    InvestorPlace - Stock Market News, Stock Advic...
12    InvestorPlace - Stock Market News, Stock Advic...
13    InvestorPlace - Stock Market News, Stock Advic...
14    InvestorPlace - Stock Market News, Stock Advic...
15    InvestorPlace - Stock Market News, Stock Advic...
16    InvestorPlace - Stock Market News, Stock Advic...
17    InvestorPlace - Stock Market News, Stock A

In [6]:
df.to_csv('stockNews.csv')
p_df.to_csv('paragraph.csv')